In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import vnstock
from datetime import datetime

In [5]:
def MA_value(day, index, df):
    MA_v = 0
    for i in range(index-day+1, index+1):
        MA_v = MA_v + df['price'][i]
    MA_v = MA_v/day
    return MA_v

def check_condition_valid(conditions, row):
    for condition in conditions:
        if row['MA'+str(condition)] == 0:
            return False
    return True

def check_conditions_actived(conditions, row):
    for condition in conditions:
        if row['price'] < row['MA'+str(condition)]:
            return False
    return True

def check_conditions_unactived(conditions, row):
    for condition in conditions:
        if row['price'] > row['MA'+str(condition)]:
            return False
    return True

In [6]:
def strategy_function(stock, start_date, end_date):
    df = vnstock.stock_historical_data(stock, start_date, end_date, "1D", 'stock')
    features = ['close']
    df = df[features]
    df.rename(columns={'close': 'price'}, inplace=True)

    MA_day_list = [5, 10, 20, 50, 200]
    for MA_day in MA_day_list:
        df['MA'+str(MA_day)] = ""
        for index in range(len(df)):
            if index < MA_day:
                df['MA'+str(MA_day)][index] = 0
            else:
                df['MA'+str(MA_day)][index] = MA_value(MA_day, index, df)

    command = pd.DataFrame({'buy_index':[], 'buy_price':[], 'sell_index':[], 'sell_price':[], 'profit_(%)':[], 'holding_days':[]})
    conditions = [5, 10, 20, 50, 200]
    for index, row in df.iterrows():
        if check_condition_valid(conditions, row) == False:
            continue
        elif check_conditions_actived(conditions, row):
            new_command = {'buy_index': index, 'buy_price': row['price']}
            command = pd.concat([command, pd.DataFrame([new_command])], ignore_index=True)
    
    command_index = 0
    while command_index < len(command):
        for i in range(int(command.iloc[command_index]['buy_index']), len(df)):
            if check_conditions_unactived(conditions, df.iloc[i]):
                buy_price = command.iloc[command_index]['buy_price']
                sell_price = df.iloc[i]['price']
                command.loc[command_index, 'sell_index'] = i
                command.loc[command_index, 'sell_price'] = sell_price
                command.loc[command_index, 'profit_(%)'] = round((sell_price-buy_price)/buy_price*100, 2)
                command.loc[command_index, 'holding_days'] = i - command.iloc[command_index]['buy_index']
                break
        command_index = command_index + 1
    
    command = command.drop(command[command.duplicated(subset=['sell_index'])].index)
    command = command.reset_index(drop=True)

    last_command_index = len(command) - 1 # we can't use -1 because it will create a new command with index -1
    if pd.isna(command.iloc[last_command_index]['sell_index']):
        command.loc[last_command_index, 'sell_index'] = len(df) - 1
        command.loc[last_command_index, 'sell_price'] = df.iloc[-1]['price']
        command.loc[last_command_index, 'profit_(%)'] = round((df.iloc[-1]['price']-command.iloc[last_command_index]['buy_price'])/command.iloc[last_command_index]['buy_price']*100, 2)
        command.loc[last_command_index, 'holding_days'] = len(df) - 1 - command.iloc[-1]['buy_index']
        
    total_command = len(command)
    total_holding_days = int(command['holding_days'].sum())
    average_holding_days = round(total_holding_days/total_command, 2)
    total_profit = round(command['profit_(%)'].sum(), 2)
    average_profit = round(total_profit/total_command, 2)

    result = {'stock': stock, 'total_command': total_command, 'total_holding_days': total_holding_days, 'average_holding_days': average_holding_days, 'total_profit': total_profit, 'average_profit': average_profit}
    return result

In [ ]:
start_date = "2016-01-01"
end_date = "2023-08-18"
start_datetime = datetime.strptime(start_date, '%Y-%m-%d')
end_datetime = datetime.strptime(end_date, '%Y-%m-%d')
duration = (end_datetime - start_datetime).days
industry = vnstock.industry_analysis("ACB", lang='vi').columns
print(industry)

In [8]:
industry_df = pd.DataFrame({'stock': [], 'total_command': [], 'total_holding_days': [], 'average_holding_days': [], 'total_profit': []})

In [ ]:
for stock in industry:
    result = strategy_function(stock, start_date, end_date)
    industry_df = pd.concat([industry_df, pd.DataFrame([result])], ignore_index = True)

In [10]:
industry_df = industry_df.sort_values(by='total_profit', ascending=False)
industry_df = industry_df.reset_index(drop=True)

In [ ]:
industry_df

In [22]:
def high_low(industry_df, feature):
    np_array = np.array(industry_df[feature].values)
    mean = round(np.mean(np_array), 2)
    high = round(mean + np.std(np_array), 2)
    low = round(mean - np.std(np_array), 2)
    return [high, mean, low]

In [ ]:
fig, ax1 = plt.subplots(figsize=(18, 7))

sns.barplot(data=industry_df, x='stock', y='total_profit', color='#69b3a2')
ax1.set_xlabel('Stock', fontsize=14)
ax1.set_ylabel('Total Profit (%)', fontsize=14)

for index, row in industry_df.iterrows():
    plt.text(index, row['total_profit'], str(row['total_profit']) + '%', ha='center', va='bottom')
    if index < 9:
        plt.text(index, row['total_profit'] / 2, str(round(row['total_profit']/(duration/365), 2)) + '% per year', rotation='vertical', ha='center', va='center')
ax2 = ax1.twinx()
sns.lineplot(data=industry_df, x='stock', y='total_command', marker='o', color='tab:blue', ax=ax2)
ax2.set_ylabel('Total Command', fontsize=14)

high = high_low(industry_df, "total_profit")[0]
mean = high_low(industry_df, "total_profit")[1]
low = high_low(industry_df, "total_profit")[2]

ax1.axhline(y=high, color='red', linestyle='-', linewidth=3, label='high')
ax1.axhline(y=mean, color='green', linestyle='-', linewidth=3, label='mean')
ax1.axhline(y=low, color='blue', linestyle='-', linewidth=3, label='low')

plt.title("Investment performance according to MA tactics of the Banking Industry", fontsize=18)
plt.show()

In [13]:
start_date = "2016-01-01"
end_date = "2023-08-18"
start_datetime = datetime.strptime(start_date, '%Y-%m-%d')
end_datetime = datetime.strptime(end_date, '%Y-%m-%d')
duration = (end_datetime - start_datetime).days
vn30 = ["ACB", "BCM", "BID", "BVH", "CTG", "FPT", "GAS", "GVR", "HDB", "HPG", "MBB", "MSN", "MWG", "PLX", "POW", "SAB", "SHB", "SSB", "SSI", "STB",
        "TCB", "TPB", "VCB", "VHM", "VIB", "VIC", "VJC", "VNM", "VPB", "VRE"]
vn30_df = pd.DataFrame({'stock': [], 'total_command': [], 'total_holding_days': [], 'average_holding_days': [], 'total_profit': []})

In [ ]:
for stock in vn30:
    result = strategy_function(stock, start_date, end_date)
    vn30_df = pd.concat([vn30_df, pd.DataFrame([result])], ignore_index = True)

vn30_df = vn30_df.sort_values(by='total_profit', ascending=False)
vn30_df = vn30_df.reset_index(drop=True)

In [ ]:
fig, ax1 = plt.subplots(figsize=(18, 9))

sns.barplot(data=vn30_df.loc[:14], x='stock', y='total_profit', color='#69b3a2')
ax1.set_xlabel('Stock', fontsize=14)
ax1.set_ylabel('Total Profit (%)', fontsize=14)

for index, row in vn30_df.iterrows():
    if index <= 14:
        plt.text(index, row['total_profit'], str(row['total_profit']) + '%', ha='center', va='bottom')
        plt.text(index, row['total_profit'] / 2, str(round(row['total_profit']/(duration/365), 2)) + '% per year', rotation='vertical', ha='center', va='center')

high = high_low(vn30_df, "total_profit")[0]
mean = high_low(vn30_df, "total_profit")[1]
low = high_low(vn30_df, "total_profit")[2]

ax1.axhline(y=high, color='red', linestyle='-', linewidth=3, label='high')
ax1.axhline(y=mean, color='green', linestyle='-', linewidth=3, label='mean')
ax1.axhline(y=low, color='blue', linestyle='-', linewidth=3, label='low')

plt.title("Investment performance according to MA tactics of the VN30", fontsize=18)
plt.show()